## Imports
This notebook should be in the root directory of `single-cell-data-portal`.

In [ ]:
from backend.wmg.api.v2 import *
import tiledb
import numpy as np
import json
import pandas as pd

## Load local snapshot 

In [ ]:
sn = "local-snapshot-name"
snapshot = WmgSnapshot(snapshot_identifier="",
    expression_summary_cube=tiledb.open(f'{sn}/expression_summary'),
    marker_genes_cube=tiledb.open(f'{sn}/marker_genes'),
    expression_summary_default_cube=tiledb.open(f'{sn}/expression_summary_default'),
    expression_summary_fmg_cube=tiledb.open(f'{sn}/expression_summary_fmg'),                       
    cell_counts_cube=tiledb.open(f'{sn}/cell_counts'),
    cell_type_orderings=pd.read_json(f'{sn}/cell_type_orderings.json'),
    primary_filter_dimensions=json.load(open(f'{sn}/primary_filter_dimensions.json','r')),
    dataset_to_gene_ids=json.load(open(f'{sn}/dataset_to_gene_ids.json','r')), 
    filter_relationships=json.load(open(f'{sn}/filter_relationships.json','r')))

## Define your request (can be retrieved from chrome Network devtools)

In [3]:
request = {'filter': {'dataset_ids': [],
  'development_stage_ontology_term_ids': [],
  'disease_ontology_term_ids': [],
  'gene_ontology_term_ids': ['ENSG00000251562'],
  'organism_ontology_term_id': 'NCBITaxon:9606',
  'self_reported_ethnicity_ontology_term_ids': [],
  'sex_ontology_term_ids': [],
  'publication_citations': []},
 'is_rollup': True}

## Copy contents of `backend.wmg.api.v2.query` into the below cell

In [ ]:
# request = connexion.request.json <-- hardcoded above
is_rollup = request.get("is_rollup", True)
compare = request.get("compare", None)

if compare:
    compare = find_dimension_id_from_compare(compare)

criteria = WmgQueryCriteriaV2(**request["filter"])

# with ServerTiming.time("query and build response"):
# snapshot: WmgSnapshot = load_snapshot(
#     snapshot_schema_version=WMG_API_SNAPSHOT_SCHEMA_VERSION,
#     explicit_snapshot_id_to_load=WMG_API_FORCE_LOAD_SNAPSHOT_ID,
# )

cube_query_params = WmgCubeQueryParams(
    cube_query_valid_attrs=READER_WMG_CUBE_QUERY_VALID_ATTRIBUTES,
    cube_query_valid_dims=READER_WMG_CUBE_QUERY_VALID_DIMENSIONS)

q = WmgQuery(snapshot, cube_query_params)
default = snapshot.expression_summary_default_cube is not None and compare is None
for dim in criteria.dict():
    if len(criteria.dict()[dim]) > 0 and depluralize(dim) in expression_summary_non_indexed_dims:
        default = False
        break

expression_summary = (
    q.expression_summary_default(criteria)
    if default
    else q.expression_summary(criteria, compare_dimension=compare)
)

cell_counts = q.cell_counts(criteria, compare_dimension=compare)
if expression_summary.shape[0] > 0 or cell_counts.shape[0] > 0:
    group_by_terms = ["tissue_ontology_term_id", "cell_type_ontology_term_id", compare] if compare else None

    gene_expression_df, cell_counts_grouped_df = get_dot_plot_data(
        expression_summary, cell_counts, group_by_terms
    )
    if is_rollup:
        rolled_gene_expression_df, rolled_cell_counts_grouped_df = rollup(
            gene_expression_df, cell_counts_grouped_df
        )

    response = (   #jsonify(
        dict(
            snapshot_id=snapshot.snapshot_identifier,
            expression_summary=build_expression_summary(gene_expression_df, rolled_gene_expression_df, compare),
            term_id_labels=dict(
                genes=build_gene_id_label_mapping(criteria.gene_ontology_term_ids),
                cell_types=build_ordered_cell_types_by_tissue(
                    rolled_cell_counts_grouped_df, snapshot.cell_type_orderings, compare
                ),
            ),
        )
    )
else:  # no data, return empty json
    response = (   #jsonify(
        dict(snapshot_id=snapshot.snapshot_identifier, expression_summary={}, term_id_labels={})
    )
# return response

## Explore results and iterate
Explore `response` to assess whether the endpoint succeeded or not. If any dependent functions in `backend.wmg.api.v2` require modification, copy them into this Jupyter notebook with the updates and run the above cell again.